In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import pickle
import joblib

import pandas as pd
import numpy as np
import duckdb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

import mlflow
from mlflow.tracking import MlflowClient


In [2]:
sys.path.append('../../..')
from connection_params import *

conn = duckdb.connect(f"md:my_db?motherduck_token={duckdb_token}")
cur = conn.cursor()

In [3]:
_input_query = f"""

select *
from my_db.main.BINANCE
order by timestamp desc

"""

df = pd.read_sql(_input_query, conn)
print(df.shape)
print(df.info())
df.head()

(993, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pair_ID    993 non-null    object 
 1   Timestamp  993 non-null    object 
 2   Open       993 non-null    float64
 3   High       993 non-null    float64
 4   Low        993 non-null    float64
 5   Close      993 non-null    float64
 6   Volume     993 non-null    float64
dtypes: float64(5), object(2)
memory usage: 54.4+ KB
None


,Pair_ID,Timestamp,Open,High,Low,Close,Volume
0,NEAR/USDT,2025-09-19 07:00:00+07:00,3.176,3.277,3.176,3.224,3.810103e+06
1,NEAR/USDT,2025-09-18 07:00:00+07:00,2.824,2.854,2.803,2.850,1.061986e+06
2,NEAR/USDT,2025-09-17 07:00:00+07:00,2.708,2.713,2.687,2.692,3.627556e+05
3,NEAR/USDT,2025-09-16 07:00:00+07:00,2.626,2.633,2.611,2.613,3.707031e+05
4,NEAR/USDT,2025-09-15 07:00:00+07:00,2.705,2.729,2.674,2.710,6.998903e+05


In [4]:
print(df.Timestamp.min())
print(df.Timestamp.max())

2023-01-01 07:00:00+07:00
2025-09-19 07:00:00+07:00


In [5]:
df['Timestamp'] = df['Timestamp'].str.split(' ', expand=True)[0]
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Pair_ID    993 non-null    object        
 1   Timestamp  993 non-null    datetime64[ns]
 2   Open       993 non-null    float64       
 3   High       993 non-null    float64       
 4   Low        993 non-null    float64       
 5   Close      993 non-null    float64       
 6   Volume     993 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 54.4+ KB
None


,Pair_ID,Timestamp,Open,High,Low,Close,Volume
0,NEAR/USDT,2025-09-19,3.176,3.277,3.176,3.224,3.810103e+06
1,NEAR/USDT,2025-09-18,2.824,2.854,2.803,2.850,1.061986e+06
2,NEAR/USDT,2025-09-17,2.708,2.713,2.687,2.692,3.627556e+05
3,NEAR/USDT,2025-09-16,2.626,2.633,2.611,2.613,3.707031e+05
4,NEAR/USDT,2025-09-15,2.705,2.729,2.674,2.710,6.998903e+05


In [6]:
df = df.sort_values(by = 'Timestamp', ascending = True).reset_index(drop=True)
df.head()

,Pair_ID,Timestamp,Open,High,Low,Close,Volume
0,NEAR/USDT,2023-01-01,1.256,1.285,1.239,1.276,4042516.75
1,NEAR/USDT,2023-01-02,1.276,1.326,1.256,1.314,6170515.50
2,NEAR/USDT,2023-01-03,1.314,1.365,1.304,1.335,7038132.00
3,NEAR/USDT,2023-01-04,1.334,1.630,1.329,1.550,30388420.00
4,NEAR/USDT,2023-01-05,1.550,1.666,1.507,1.524,18142158.00


In [7]:
index = 5
i_5 = index - 5
tmp = df.iloc[i_5:index]['Close'].mean()
tmp

np.float64(1.3998000144958496)

In [8]:
def generate_feature(df, i: int, col_name: str):
    Pre_mean = []
    Pre_std = []
    for index, row in df.iterrows():
        if index < i:
            Pre_mean.append(0)
            Pre_std.append(0)
        else:
            i_index = index - i
            Pre_mean.append(df.iloc[i_index:index][col_name].mean())
            Pre_std.append(df.iloc[i_index:index][col_name].std())
    
    df[f'Pre_{col_name}_{i}_mean'] = Pre_mean
    df[f'Pre_{col_name}_{i}_std'] = Pre_std
    return df 


In [9]:
for day in [5, 10, 20, 30]:
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        df = generate_feature(df, day, col)

df.head()

,Pair_ID,Timestamp,Open,High,Low,Close,Volume,Pre_Open_5_mean,Pre_Open_5_std,Pre_High_5_mean,...,Pre_Open_30_mean,Pre_Open_30_std,Pre_High_30_mean,Pre_High_30_std,Pre_Low_30_mean,Pre_Low_30_std,Pre_Close_30_mean,Pre_Close_30_std,Pre_Volume_30_mean,Pre_Volume_30_std
0,NEAR/USDT,2023-01-01,1.256,1.285,1.239,1.276,4042516.75,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NEAR/USDT,2023-01-02,1.276,1.326,1.256,1.314,6170515.50,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NEAR/USDT,2023-01-03,1.314,1.365,1.304,1.335,7038132.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NEAR/USDT,2023-01-04,1.334,1.630,1.329,1.550,30388420.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NEAR/USDT,2023-01-05,1.550,1.666,1.507,1.524,18142158.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# BackTesting

# train, test
from datetime import datetime 

train = df[df.Timestamp < datetime(year = 2025, month = 1, day = 1)]
test = df[df.Timestamp >= datetime(year = 2025, month = 1, day = 1)]
print(train.shape)
print(test.shape)

(731, 47)
(262, 47)


In [11]:
feature = train.columns[7:]
label = 'Close'

X_train, y_train = train[feature].copy(), train[label]
X_test, y_test = train[feature].copy(), train[label]

In [12]:
# Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("num", 'passthrough', feature)
    ]
)

params = {
    'objective': 'reg:squarederror', 
    'random_state': 42
}

# Create a pipeline with preprocessing and regression
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    # ("regressor", LinearRegression())
    ("regressor", XGBRegressor())
])

pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [13]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

metrics = {
        'train_R2': r2_score(y_train_pred, y_train) * 100,
        'train_MAPE': mean_absolute_percentage_error(y_train_pred, y_train) * 100,
        'train_MAE': mean_absolute_error(y_train_pred, y_train) * 100,
        'test_R2': r2_score(y_test_pred, y_test) * 100,
        'test_MAPE': mean_absolute_percentage_error(y_test_pred, y_test) * 100,
        'test_MAE': mean_absolute_error(y_test_pred, y_test) * 100 
}
metrics

{'train_R2': 99.99772437294563,
 'train_MAPE': 0.15618365853717084,
 'train_MAE': 0.32170183355276627,
 'test_R2': 99.99772437294563,
 'test_MAPE': 0.15618365853717084,
 'test_MAE': 0.32170183355276627}

In [14]:
# 1. Setup MLflow tracking & experiment
# =====================================================
TRACKING_URI = "file:///C:/Users/Admin/Documents/DataScience/DataOps/MLOps/modeling/stock_model/mlflow"
EXPERIMENT_NAME = "stock_experiment"
REGISTERED_MODEL_NAME = "StockModel"

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

client = MlflowClient()

2025/09/20 00:10:21 INFO mlflow.tracking.fluent: Experiment with name 'stock_experiment' does not exist. Creating a new experiment.


In [15]:
# =====================================================
# 2. Train and log a model
# =====================================================
run_name = "stock_model_demo"

with mlflow.start_run(run_name=run_name) as run:
    # --- log hyperparameters & metrics
    mlflow.log_params(params)       # <-- your dict of params
    mlflow.log_metrics(metrics)     # <-- your dict of metrics

    # --- save the model as artifact of this run
    artifact_path = "model"
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        name=artifact_path,
        input_example=X_test,
    )

In [16]:
# =====================================================
# 3. Search for the best run by metric (e.g. lowest rmse)
# =====================================================
runs_df = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    # order_by=["metrics.test_MAPE DESC"],      # sort DESC by MAPE
    order_by=["metrics.test_MAPE ASC"],      # sort DESC by MAPE
    max_results=1
)

best_run_id = runs_df.iloc[0].run_id
best_mape = runs_df.iloc[0]["metrics.test_MAPE"]
print(f"Best run: {best_run_id} (MAPE={best_mape})")

Best run: f94e47bc97ec4275b31c034cd8e6d079 (MAPE=0.15618365853717084)


In [17]:
# =====================================================
# 4. Register that best run as a model
# =====================================================
model_uri = f"runs:/{best_run_id}/{artifact_path}"
registration = mlflow.register_model(model_uri, REGISTERED_MODEL_NAME)

Successfully registered model 'StockModel'.
2025/09/20 00:10:26 WARNING mlflow.tracking._model_registry.fluent: Run with id f94e47bc97ec4275b31c034cd8e6d079 has no artifacts at artifact path 'model', registering model based on models:/m-72bbfc04bc7b4dbf8ede2c2a526d83a5 instead
Created version '1' of model 'StockModel'.


In [18]:
# client.update_model_version(
#     name=REGISTERED_MODEL_NAME,
#     version=registration.version,
#     description=f"Best RMSE={best_rmse:.4f} from run {best_run_id}",
# )
# client.transition_model_version_stage(
#     name=REGISTERED_MODEL_NAME,
#     version=registration.version,
#     stage="Production",
# )

# print(
#     f"✅ Best model registered: {REGISTERED_MODEL_NAME} v{registration.version}, "
#     f"RMSE={best_rmse:.4f}, stage=Production"
# )

# # =====================================================
# # 5. Load the Production model
# # =====================================================
# # Option A: load by stage (always gives the latest model in that stage)
# prod_uri = f"models:/{REGISTERED_MODEL_NAME}/Production"
# best_model = mlflow.sklearn.load_model(prod_uri)

In [19]:
registration.version

1

In [20]:
# Option B: load a specific version
version_uri = f"models:/{REGISTERED_MODEL_NAME}/{registration.version}"

best_model = mlflow.sklearn.load_model(version_uri)

# Use the model
y_pred = best_model.predict(X_test)
print("Predictions:", y_pred[:5])

Predictions: [1.3997878 1.3997878 1.3997878 1.3997878 1.3997878]


In [21]:
y_test[:5]

0    1.276
1    1.314
2    1.335
3    1.550
4    1.524
Name: Close, dtype: float64